In [1]:
import pandas
from pandas import read_excel, date_range, DatetimeIndex, DataFrame, to_numeric, concat, to_datetime
from pandas.plotting import lag_plot, autocorrelation_plot
from pandas.tseries.offsets import DateOffset

import numpy
from numpy import logical_not

import statistics
import dateutil.relativedelta
from dateutil.relativedelta import relativedelta

from prophet import Prophet

import calendar
from calendar import monthrange

from datetime import datetime, timedelta
from matplotlib import pyplot

from sklearn.metrics import mean_absolute_error, mean_squared_error

from math import sqrt

In [2]:
"""
Pedro Tobarra 20210907:
Modificación del notebook '20210630 seguro_vehiculo_78_v01_def.ipynb' para meter el código en una función 
de python que luego pueda ser implementada en un fichero '.py' para su integración con el backend del PFM
"""

"\nPedro Tobarra 20210907:\nModificación del notebook '20210630 seguro_vehiculo_78_v01_def.ipynb' para meter el código en una función \nde python que luego pueda ser implementada en un fichero '.py' para su integración con el backend del PFM\n"

In [3]:
# cambiamos el valor de estos 4 parametros para que nos muestre dataframes truncados o sin truncar
pandas.set_option('display.max_rows', 12)
# pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', None)

In [4]:
# Load data using read_excel
transacciones_df = read_excel('20210513 mmelero (249236).xlsx', sheet_name='Hoja1')
transacciones_df

,Fecha transacción,Importe,ID Categoría,Nombre categoría,Nivel categoría,iD categoría padre,Nombre categoría padre,Proveedor,Marca,Sector,Financiero,Transferencia,Unnamed: 12,Unnamed: 13,ID,BALANCE,BALANCE_DATE
0,2020-07-24,-1.20,84.0,parking_84,2.0,5.0,vehiculos_5,NaN,NaN,NaN,False,False,NaN,NaN,249236.0,1478.98,2020-07-26
1,2020-07-23,-4.50,117.0,regalos_celebraciones_117,2.0,8.0,gastos_personales_8,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
2,2020-07-23,-13.53,70.0,supermercados_70,2.0,4.0,alimentacion_4,SPAR,SPAR,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
3,2020-07-22,-51.40,222.0,adsl_222,3.0,521.0,Comunicaciones y TV,Jazz Telecom,Jazztel,Compañías telecomunicaciones,False,False,NaN,NaN,NaN,NaN,NaT
4,2020-07-22,-11.00,70.0,supermercados_70,2.0,4.0,alimentacion_4,Mercadona,Mercadona,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,2016-09-05,1533.02,18.0,retribucion_liquida_18,3.0,576.0,Nómina,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
1483,2016-09-05,-617.47,126.0,liquidacion_tarjeta_126,3.0,478.0,tarjetas_credito_478,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaT
1484,2016-09-05,-138.08,125.0,cuotas_125,3.0,127.0,otros_pasivos_127,NaN,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaT
1485,2016-09-01,-32.22,90.0,seguro_salud_90,2.0,6.0,salud_6,Sanitas S.A. de Seguros,Sanitas,Seguros de salud,True,False,NaN,NaN,NaN,NaN,NaT


In [59]:
def seguro_vehiculo_78(transacciones_df):
    
    # nos quedamos con la fecha en que nos dan los datos de las transacciones
    last_date_tstamp = transacciones_df.loc[0,'BALANCE_DATE']
    
    # nos quedamos con las transacciones de la categoría 'seguro_vehiculo_78'
    transacciones_78_df = transacciones_df[transacciones_df['ID Categoría'] == 78.0]
    del transacciones_df
    
    # nos quedamos con categoría desde 'Fecha transacción' hasta 'Importe'
    transacciones_78_df = transacciones_78_df.iloc[:, 0:2]
    
    # renombramos columnas
    transacciones_78_df.rename(columns={'Fecha transacción': 'FECHA', 'Importe': 'IMPORTE'}, inplace=True)
    
    # ordenamos las fechas por orden ascendente
    transacciones_78_df.sort_values(by=['FECHA'], ascending=True, inplace=True, ignore_index=True)
    
    # vamos a agrupar los valores y sumarlos por fecha para agrupar cargos distintos realizados el mismo dia
    # (por si acaso)
    transacciones_78_df = transacciones_78_df.groupby(['FECHA']).sum()
    
    # para que los datos sean más fáciles de interpretar vamos a hacerlos todos positivos multiplicándolos por '-1'
    transacciones_78_df['IMPORTE'] = -transacciones_78_df['IMPORTE']
    
    # EMPEZAMOS POR HACER EL DATAFRAME DE LOS IMPORTES
    # vamos a rellenar las missing dates con el ultimo valor válido
    transacciones_78_importe_df = transacciones_78_df.copy()
    
    # hacemos una columna con la fecha del indice
    transacciones_78_importe_df['FECHA'] = transacciones_78_importe_df.index
    
    idx = date_range(start=transacciones_78_importe_df.FECHA.min(), end=transacciones_78_importe_df.FECHA.max())
    
    # pasamos el indice de transacciones_78_importe_df a formato DatetimeIndex
    transacciones_78_importe_df.index = DatetimeIndex(transacciones_78_importe_df.index)
    
    # rellenamos las missing dates en el indice
    transacciones_78_importe_df = transacciones_78_importe_df.reindex(idx, fill_value='NaN')
    
    # hacemos drop de la columna FECHA
    transacciones_78_importe_df.drop(columns='FECHA', inplace=True)
    
    # pasamos IMPORTE a formato 'numeric'
    transacciones_78_importe_ser = transacciones_78_importe_df.T.squeeze()
    transacciones_78_importe_ser = to_numeric(transacciones_78_importe_ser, errors='coerce')
    transacciones_78_importe_df = DataFrame(transacciones_78_importe_ser)
    del transacciones_78_importe_ser
    
    # rellenamos los NaN con el ultimo valor numérico anterior
    transacciones_78_importe_df['IMPORTE'].fillna(method='ffill', inplace=True)
    
    # HACEMOS EL DATAFRAME DEL IMPORTE
    # para el dataframe del importe cojemos el ultimo año de datos y lo repetimos 3 veces hacia atras
    transacciones_78_importe_past_df = transacciones_78_importe_df.copy()
    
    # hacemos una columna con la fecha a partir del índice
    transacciones_78_importe_past_df['FECHA'] = transacciones_78_importe_past_df.index
    
    # vamos a repetir el ultimo año 3 veces hacia atras
    end_date = transacciones_78_importe_past_df.FECHA.max()    
    start_date = end_date - relativedelta(years=1)
    
    # hacemos una mascara con el ultimo año de transacciones
    mask = (transacciones_78_importe_past_df['FECHA'] > start_date) & (transacciones_78_importe_past_df['FECHA'] 
                                                                       <= end_date)
    
    # nos quedamos con el ultimo año de transacciones aplicando la mascara
    transacciones_78_importe_plus0_df = transacciones_78_importe_past_df.loc[mask]
    
    # hacemos el dataframe de 1 año atras
    transacciones_78_importe_minus1_df = transacciones_78_importe_plus0_df.copy()
    
    # restamos 1 año en columna fecha
    transacciones_78_importe_minus1_df['FECHA'] = transacciones_78_importe_plus0_df['FECHA'] - DateOffset(years=1)
    
    # hacemos el dataframe de 2 años atras
    transacciones_78_importe_minus2_df = transacciones_78_importe_plus0_df.copy()
    
    # restamos 2 años en columna fecha
    transacciones_78_importe_minus2_df['FECHA'] = transacciones_78_importe_plus0_df['FECHA'] - DateOffset(years=2)
    
    # reseteamos los indices de los 3 dataframes
    transacciones_78_importe_plus0_df.reset_index(drop=True, inplace=True)
    transacciones_78_importe_minus1_df.reset_index(drop=True, inplace=True)
    transacciones_78_importe_minus2_df.reset_index(drop=True, inplace=True)
    
    # a continuacion contatenamos los 3 dataframes
    transacciones_78_importe_minus2_df = transacciones_78_importe_minus2_df.append(
        transacciones_78_importe_minus1_df, ignore_index=True)
    
    transacciones_78_importe_minus2_df = transacciones_78_importe_minus2_df.append(
        transacciones_78_importe_plus0_df, ignore_index=True)
    
    del transacciones_78_importe_past_df
    
    transacciones_78_importe_past_df = transacciones_78_importe_minus2_df.copy()
    
    del transacciones_78_importe_minus2_df
    del transacciones_78_importe_minus1_df
    del transacciones_78_importe_plus0_df    
    
    # HACEMOS EL DATAFRAME DEL DIA DE PAGO DEL SEGURO DEL COCHE HASTA EL AÑO SIGUIENTE AL DE LAST_DATE_TSTAMP
    
    """
    # extendemos dataframe del dia de pago del seguro hasta 1 año despues de la fecha de la ultima transaccion en
    # cuenta
    target_year = 1 + last_date_tstamp.year
    
    # hacemos la diferencia entre target_year y transacciones_78_importe_past_df.iloc[-1]['FECHA'].year para saber
    # cuantos años tenemos q extender el dataframe anterior
    numberOf_future_years = target_year - transacciones_78_df.index[-1].year
    
    # transacciones_78_forecast_df sera el dataframe donde guardaremos las futuras fechas de cobro 
    # (y las pasadas reales)
    transacciones_78_forecast_df = transacciones_78_df.copy()
    
    # calculo el 1er y ultimo dia del ultimo año
    start_date_dtime = datetime(transacciones_78_forecast_df.index[-1].year, 1, 1)
    end_date_dtime = datetime(transacciones_78_forecast_df.index[-1].year, 12, 31)
    
    # construyo la condicion para seleccionar las fechas entre el 1er y ultimo dia del ultimo año
    after_start_date = transacciones_78_forecast_df.index >= start_date_dtime
    before_end_date = transacciones_78_forecast_df.index <= end_date_dtime
    between_two_dates = after_start_date & before_end_date
    
    # construyo un dataframe solo con las fechas del ultimo año
    transacciones_78_nextYear_df = transacciones_78_forecast_df.loc[between_two_dates]
    
    # añadimos 1 año al indice
    transacciones_78_nextYear_df.index += pandas.offsets.DateOffset(years=1)
    
    # añadimos las filas de transacciones_78_nextYear_df a las de transacciones_78_forecast_df
    transacciones_78_forecast_df = transacciones_78_forecast_df.append(transacciones_78_nextYear_df)
    
    # decrementamos el contador de años futuros
    numberOf_future_years -= 1
    
    # borramos el dataframe q tiene solo las transacciones del ultimo año
    del transacciones_78_nextYear_df
    
    # repetimos las operaciones anteriores para añadir un año más hata el 2021
    
    # calculo el 1er y ultimo dia del ultimo año
    start_date_dtime = datetime(transacciones_78_forecast_df.index[-1].year, 1, 1)
    end_date_dtime = datetime(transacciones_78_forecast_df.index[-1].year, 12, 31)
    
    # construyo la condicion para seleccionar las fechas entre el 1er y ultimo dia del ultimo año
    after_start_date = transacciones_78_forecast_df.index >= start_date_dtime
    before_end_date = transacciones_78_forecast_df.index <= end_date_dtime
    between_two_dates = after_start_date & before_end_date
    
    # construyo un dataframe solo con las fechas del ultimo año
    transacciones_78_nextYear_df = transacciones_78_forecast_df.loc[between_two_dates]
    
    # añadimos 1 año al indice
    transacciones_78_nextYear_df.index += pandas.offsets.DateOffset(years=1)
    
    # añadimos las columnas de transacciones_78_nextYear_df a las de transacciones_78_forecast_df
    transacciones_78_forecast_df = transacciones_78_forecast_df.append(transacciones_78_nextYear_df)
    
    # decrementamos el contador de años futuros
    numberOf_future_years -= 1
    
    # borramos el dataframe q tiene solo las transacciones del ultimo año
    del transacciones_78_nextYear_df
    """
    
    """
    Y con esto ya tenemos el dataframe hecho para predecir próximas fechas de cobro e incluso también los
    respectivos importes de cobro (proyectando los del ultimo año con datos) si no queremos usar prophet para
    calcularlos
    """
    
    # ahora vamos a hacer lo mismo que en el grupo de transacciones anteriores pero con un 'for' de forma automatica

    # extendemos dataframe del dia de pago del seguro hasta 1 año despues de la fecha de la ultima transaccion en
    # cuenta
    target_year = 1 + last_date_tstamp.year

    # hacemos la diferenca entre target_year y transacciones_78_importe_past_df.iloc[-1]['FECHA'].year para saber
    # cuantos años tenemos q extender el dataframe anterior
    numberOf_future_years = target_year - transacciones_78_df.index[-1].year

    # transacciones_78_forecast_df sera el dataframe donde guardaremos las futuras fechas de cobro (y las pasadas
    # reales)
    transacciones_78_forecast_df = transacciones_78_df.copy()

    while numberOf_future_years > 0:
        # calculo el 1er y ultimo dia del ultimo año
        start_date_dtime = datetime(transacciones_78_forecast_df.index[-1].year, 1, 1)
        end_date_dtime = datetime(transacciones_78_forecast_df.index[-1].year, 12, 31)

        # construyo la condicion para seleccionar las fechas entre el 1er y ultimo dia del ultimo año
        after_start_date = transacciones_78_forecast_df.index >= start_date_dtime
        before_end_date = transacciones_78_forecast_df.index <= end_date_dtime
        between_two_dates = after_start_date & before_end_date

        # construyo un dataframe solo con las fechas del ultimo año
        transacciones_78_nextYear_df = transacciones_78_forecast_df.loc[between_two_dates]	

        # añadimos 1 año al indice
        transacciones_78_nextYear_df.index += pandas.offsets.DateOffset(years=1)

        # añadimos las filas de transacciones_78_nextYear_df a las de transacciones_78_forecast_df
        transacciones_78_forecast_df = transacciones_78_forecast_df.append(transacciones_78_nextYear_df)

        # decrementamos el contador de años futuros
        numberOf_future_years -= 1

        # borramos el dataframe q tiene solo las transacciones del ultimo año
        del transacciones_78_nextYear_df
    
    # con el dataframe 'transacciones_78_forecast_df' ya tenemos estimados el importe y las fechas de cobro
    # mediante la 'cuenta de la vieja'    
    
    # A CONTINUACION CON LOS DATOS ANTERIORES VAMOS A HACER LA PARTE DEL ALGORITMO QUE HACE LAS PREDICCIONES
    # pedimos la fecha al usuario (en producción tomamos la fecha del sistema)
#     year = input('year: ')
#     month = input('month: ')
#     day = input('day: ')
    
    year = 2020
    month = 7
    day = 26
    
    # pasamos la fecha a string
    system_date_str = str(year) + '-' + str(month) + '-' + str(day)
    
    # pasamos la fecha al formato datetime
    system_date_obj = datetime.strptime(system_date_str, '%Y-%m-%d')
    
    # en transacciones_78_forecast_df tenemos la fecha de los próximos recibos y su importe (proyectados del
    # pasado) hasta 1 año después de la fecha del sistema
    
    # hacemos un bucle desde la fecha del sistema; incrementándola hasta que encontremos la fecha del próximo
    # recibo en el indice transacciones_78_forecast_df
    
    # calculamos el rango de fechas del bucle for para buscar las fechas de los próximos recibos
    idx = date_range(start=system_date_obj, end=transacciones_78_forecast_df.index.max())
    
    # inicializamos la variable de la fecha del próximo recibo
    next_bill_date_obj = system_date_obj
    
    # buscamos la fecha del próximo recibo
    for date_obj in idx:
        if date_obj in transacciones_78_forecast_df.index:
            next_bill_date_obj = date_obj
            break

    # obtenemos el importe del recibo del dataframe transacciones_78_forecast_df para next_bill_date_obj
    next_bill_amount = float(transacciones_78_forecast_df.loc[next_bill_date_obj, 'IMPORTE'])
    
    # chequeamos si la fecha del próximo recibo está en el mes siguiente al de la petición para dar aviso
    next_month_bool = False
    if next_bill_date_obj.month - system_date_obj.month == 1:
        next_month_bool = True
        
    # chequeamos si hay un recibo el mismo dia (+/- 3 dias) y cantidad next_bill_date_obj en transacciones_78_df 
    # y si la diferencia entre next_bill_date_obj y esta fecha encontrada es igual o menor a 1 año para dar la
    # predicción como real
    last_year_bool = False
    if next_bill_amount in transacciones_78_df['IMPORTE'].tolist():
        last_bill_date_obj = max(transacciones_78_df.index[transacciones_78_df['IMPORTE'] == 
                                                           next_bill_amount].tolist())
        if abs(next_bill_date_obj.day - last_bill_date_obj.day) <= 3 & \
                next_bill_date_obj.year - last_bill_date_obj.year <= 1:
            last_year_bool = True
            
#     print()
#     print('Fecha sistema:\t\t\t\t\t', system_date_str)
    mensaje1_str = 'Fecha sistema:\t\t\t\t\t ' + system_date_str
    next_bill_date_str = next_bill_date_obj.strftime('%Y-%m-%d')
#     print('Fecha próximo recibo:\t\t\t\t', next_bill_date_str)
    mensaje2_str = 'Fecha próximo recibo:\t\t\t\t ' + next_bill_date_str
    last_bill_date_str = last_bill_date_obj.strftime('%Y-%m-%d')
#     print('Fecha último recibo equivalente (mismo importe):', last_bill_date_str)
    mensaje3_str = 'Fecha último recibo equivalente (mismo importe): ' + last_bill_date_str
#     print()
#     print('Importe próximo recibo (proyectado):\t\t', int(next_bill_amount), 'eur')
    mensaje4_str = 'Importe próximo recibo (proyectado):\t\t ' + str(int(next_bill_amount)) + ' eur'
#     print()
#     print('Aviso recibo real (anterior en último año):\t', last_year_bool)
    mensaje5_str = 'Aviso recibo real (anterior en último año):\t ' + str(last_year_bool)
#     print('Aviso recibo mes siguiente:\t\t\t', next_month_bool)
    mensaje6_str = 'Aviso recibo mes siguiente:\t\t\t ' + str(next_month_bool)
#     print('Aviso definitivo recibo mes siguiente\t\t', last_year_bool & next_month_bool)
    next_month_def_bool = last_year_bool & next_month_bool
    mensaje7_str = 'Aviso definitivo recibo mes siguiente\t\t ' + str(last_year_bool & next_month_bool)

#     print(last_date_tstamp)
#     print(transacciones_78_df)
#     print(transacciones_78_importe_df)
#     print(idx)
#     print(transacciones_78_importe_ser)
#     print(transacciones_78_importe_past_df)
#     print(end_date)
#     print(start_date)
#     print(mask)
#     print(transacciones_78_importe_plus0_df)
#     print(transacciones_78_importe_minus1_df)
#     print(transacciones_78_importe_minus2_df)
#     print(target_year)
#     print(numberOf_future_years)
#     print(transacciones_78_forecast_df)
#     print(start_date_dtime)
#     print(end_date_dtime)
#     print(transacciones_78_nextYear_df)
#     print(system_date_str)
#     print(system_date_obj)
#     print(next_bill_date_obj)
#     print(next_bill_amount)
#     print(next_month_bool)
#     print(last_year_bool)        
#     print()
    
    return system_date_obj, mensaje1_str, next_bill_date_obj, mensaje2_str, last_bill_date_obj, mensaje3_str,\
next_bill_amount, mensaje4_str, last_year_bool, mensaje5_str, next_month_bool, mensaje6_str, next_month_def_bool,\
mensaje7_str

# Load data using read_excel
transacciones_orig_df = read_excel('20210513 mmelero (249236).xlsx', sheet_name='Hoja1')

system_date, mensaje1, next_bill_date, mensaje2, last_bill_date, mensaje3, next_bill_amount_float, mensaje4,\
last_year_b, mensaje5, next_month_b, mensaje6, next_month_def_b, mensaje7 =\
seguro_vehiculo_78(transacciones_orig_df)

# print()
print(system_date)
print(mensaje1)
print(next_bill_date)
print(mensaje2)
print(last_bill_date)
print(mensaje3)
print(next_bill_amount_float)
print(mensaje4)
print(last_year_b)
print(mensaje5)
print(next_month_b)
print(mensaje6)
print(next_month_def_b)
print(mensaje7)

2020-07-26 00:00:00
Fecha sistema:					 2020-7-26
2020-10-11 00:00:00
Fecha próximo recibo:				 2020-10-11
2019-10-11 00:00:00
Fecha último recibo equivalente (mismo importe): 2019-10-11
126.1
Importe próximo recibo (proyectado):		 126 eur
True
Aviso recibo real (anterior en último año):	 True
False
Aviso recibo mes siguiente:			 False
False
Aviso definitivo recibo mes siguiente		 False
